In [4]:
# ============================================================
# DYNAMIC VRP — 30-DAY SIMULATION WITH 3-DAY COLLECTION POLICY
# CORRECTED VERSION: Collection starts on Day 1, then every 3 days
# Collection days: 1, 4, 7, 10, 13, 16, 19, 22, 25, 28
# ============================================================

import time

import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# --------------------------
# FILE PATHS
# --------------------------
file_parameters = (
    r"C:\Users\hecto\OneDrive - Universidade de Lisboa\PhDtese\TestVRPP-Determinsit_104bins\parameters.xlsx"
)
file_distances = (
    r"C:\Users\hecto\OneDrive - Universidade de Lisboa\PhDtese\TestVRPP-Determinsit_104bins\distance_104bins.xlsx"
)

output_excel_kpi = r"C:\Users\hecto\OneDrive - Universidade de Lisboa\PhDtese\TestVRPP-104bins_KPI_3DAY_POLICY.xlsx"
output_excel_flows = output_excel_kpi.replace("_KPI_", "_Flows_")
output_excel_dynamic = output_excel_kpi.replace("_KPI_", "_DYNAMIC_")
output_excel_overflow = output_excel_kpi.replace("_KPI_", "_OVERFLOW_")
output_excel_vehicles = output_excel_kpi.replace("_KPI_", "_VEHICLE_DETAILS_")

# --------------------------
# MODEL PARAMETERS
# --------------------------
C = 1.0  # Transportation cost per km
R = 0.5837  # Revenue per kg of collected waste
Omega = 0.1  # Fixed cost per vehicle usage
Q = 3500.0  # Vehicle capacity (kg)
T_days = 30  # Simulation horizon (days)
T = list(range(1, T_days + 1))

# Waste container parameters
B = 19  # Container volume parameter
vol = 2.5  # Volume multiplier

# 🎯 COLLECTION POLICY PARAMETERS
COLLECTION_FREQUENCY = 3  # Collect all bins every 3 days
FIRST_COLLECTION_DAY = 1  # Start collecting on Day 1

# Solver parameters
SOLVER_TIME_LIMIT = 28800  # 8 hours
SOLVER_MIP_GAP = 0.0001
SOLVER_THREADS = 4

# --------------------------
# READ INPUT DATA
# --------------------------
print("=" * 80)
print(" " * 25 + "READING INPUT DATA")
print("=" * 80)

df_params = pd.read_excel(file_parameters, sheet_name="Si_Ei", engine="openpyxl")
IDs_model = df_params["ID"].astype(int).tolist()
S_base = {int(r["ID"]): float(r["Si"]) for _, r in df_params.iterrows()}
E_dict = {int(r["ID"]): float(r["Ei"]) for _, r in df_params.iterrows()}
avg_ai = {int(r["ID"]): float(r["Mean"]) for _, r in df_params.iterrows()}

print(f"✓ Loaded {len(IDs_model)} bins")

df_ait = pd.read_excel(file_parameters, sheet_name="%ai_t", engine="openpyxl")
S_init = {}
ait_kg_dict = {}
for _, row in df_ait.iterrows():
    i = int(row["ID"])
    percent_S0 = float(row["Day 0"]) if "Day 0" in df_ait.columns and not pd.isna(row["Day 0"]) else 0.0
    S_init[i] = (percent_S0 / 100.0) * B * vol
    for t in T:
        col = f"Day {t}"
        ait_kg_dict[(i, t)] = (
            (float(row[col]) / 100.0 * B * vol) if col in df_ait.columns and not pd.isna(row[col]) else 0.0
        )

print("✓ Loaded daily accumulation data")

# Distance matrix
df_dist = pd.read_excel(file_distances, engine="openpyxl", index_col=0)
df_dist.index = df_dist.index.astype(int)
df_dist.columns = df_dist.columns.astype(int)

nodes_real = IDs_model
n_bins = len(nodes_real)
fictitious_node = n_bins + 1
nodes = [0] + nodes_real + [fictitious_node]

distances = {}
for i in nodes:
    for j in nodes:
        ii = 0 if i == fictitious_node else i
        jj = 0 if j == fictitious_node else j
        distances[(i, j)] = float(df_dist.loc[ii, jj])

print(f"✓ Distance matrix loaded: {len(distances)} arcs")

# --------------------------
# COMPUTE AND DISPLAY COLLECTION SCHEDULE
# --------------------------
print("\n" + "=" * 80)
print(" " * 20 + "COLLECTION SCHEDULE (3-DAY POLICY)")
print("=" * 80)

collection_days_schedule = [t for t in T if ((t - FIRST_COLLECTION_DAY) % COLLECTION_FREQUENCY == 0)]
print(f"\nCollection will occur on {len(collection_days_schedule)} days:")
print(f"Days: {collection_days_schedule}")
print("\nVerification:")
for i, day in enumerate(collection_days_schedule[:5], 1):
    gap = collection_days_schedule[i] - day if i < len(collection_days_schedule) else None
    print(
        f"  Collection {i}: Day {day}"
        + (f" → Next: Day {collection_days_schedule[i]} (gap: {gap} days)" if gap else " (last)")
    )

# --------------------------
# INITIALIZE SIMULATION STATE
# --------------------------
S_current = {i: S_base[i] for i in nodes_real}

# Storage tables
dynamic_rows = []
kpi_rows = []
flow_rows = []
overflow_daily_rows = []
vehicle_details_rows = []

print("\n" + "=" * 80)
print(" " * 20 + "STARTING 30-DAY SIMULATION")
print("=" * 80)

# ==========================
# DAILY SIMULATION LOOP
# ==========================
for t in T:
    print(f"\n{'=' * 80}")
    print(f"{'DAY ' + str(t):^80}")
    print("=" * 80)

    # --------------------------
    # Compute waste before collection
    Wit_before_dict = {i: S_current[i] + ait_kg_dict.get((i, t), 0.0) for i in nodes_real}

    total_waste = sum(Wit_before_dict.values())
    bins_at_capacity = [i for i in nodes_real if Wit_before_dict[i] >= E_dict[i]]

    print("Waste status BEFORE collection:")
    print(f"  Total waste: {total_waste:.2f} kg")
    print(f"  Bins at/over capacity: {len(bins_at_capacity)}/{n_bins}")

    # --------------------------
    # 🎯 CORRECTED: Determine if collection day (starts Day 1, then every 3 days)
    is_collection_day = (t - FIRST_COLLECTION_DAY) % COLLECTION_FREQUENCY == 0 and t >= FIRST_COLLECTION_DAY

    collected_planned_dict = {i: 1 if is_collection_day else 0 for i in nodes_real}
    planned_bins = [i for i, v in collected_planned_dict.items() if v == 1]

    # Display schedule
    if is_collection_day:
        collection_number = ((t - FIRST_COLLECTION_DAY) // COLLECTION_FREQUENCY) + 1
        days_since_last = COLLECTION_FREQUENCY if collection_number > 1 else 0
        print(f"\n🗓️  COLLECTION DAY #{collection_number}")
        print(f"   Days since last collection: {days_since_last}")
        print(f"   Planned bins: {len(planned_bins)} (ALL)")
    else:
        days_until_next = COLLECTION_FREQUENCY - ((t - FIRST_COLLECTION_DAY) % COLLECTION_FREQUENCY)
        print("\n📅 Accumulation day (no collection)")
        print(f"   Days until next collection: {days_until_next}")

    # --------------------------
    # NON-COLLECTION DAYS: Accumulate only
    if not is_collection_day:
        for i in nodes_real:
            S_before = S_current[i]
            a_it = ait_kg_dict.get((i, t), 0.0)
            Wit_before = S_before + a_it
            S_after = Wit_before
            S_current[i] = S_after

            overflow_kg = max(0, Wit_before - E_dict[i])
            overflow_daily_rows.append({"Day": t, "BinID": i, "Overflow_kg": overflow_kg})

            dynamic_rows.append(
                {
                    "Day": t,
                    "BinID": i,
                    "S_before": S_before,
                    "Daily_accumulation": a_it,
                    "Waste_before": Wit_before,
                    "Planned_collection": 0,
                    "Actual_collection": 0,
                    "Waste_after": S_after,
                    "Profit_planned": 0.0,
                    "Profit_actual": 0.0,
                    "InRoute": 0,
                    "OverflowStatus": "AT_CAPACITY" if overflow_kg > 0 else "SAFE",
                }
            )
        continue

    # --------------------------
    # COLLECTION DAY: Build VRP model
    print("\n" + "-" * 80)
    print("BUILDING VRP MODEL")
    print("-" * 80)

    model = gp.Model(f"VRP_Day_{t}")
    model.setParam("OutputFlag", 0)
    model.setParam("TimeLimit", SOLVER_TIME_LIMIT)
    model.setParam("MIPGap", SOLVER_MIP_GAP)
    model.setParam("Threads", SOLVER_THREADS)
    model.setParam("LazyConstraints", 1)

    # Decision variables
    x = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
    g = model.addVars(nodes_real, vtype=GRB.BINARY, name="g")
    f = model.addVars(nodes, nodes, vtype=GRB.CONTINUOUS, lb=0, name="f")

    # Objective
    model.setObjective(
        R * gp.quicksum(Wit_before_dict[i] * g[i] for i in nodes_real)
        - C * gp.quicksum(x[i, j] * distances[(i, j)] for i in nodes for j in nodes if i != j)
        - Omega * gp.quicksum(x[0, j] for j in nodes_real),
        GRB.MAXIMIZE,
    )

    # --------------------------
    # CONSTRAINTS

    # 1. Collect ALL bins on collection days
    model.addConstrs((g[i] == 1 for i in nodes_real), name="collect_all")

    # 2. Inflow: exactly ONE arc entering each visited bin (not from fictitious_node)
    model.addConstrs(
        (gp.quicksum(x[j, i] for j in nodes if j != i and j != fictitious_node) == g[i] for i in nodes_real),
        name="inflow",
    )

    # 3. Outflow: exactly ONE arc leaving each visited bin (not to depot 0)
    model.addConstrs(
        (gp.quicksum(x[i, j] for j in nodes if j != i and j != 0) == g[i] for i in nodes_real),
        name="outflow",
    )

    # 4. Depot balance
    model.addConstr(
        gp.quicksum(x[0, j] for j in nodes_real) == gp.quicksum(x[i, fictitious_node] for i in nodes_real),
        name="depot_balance",
    )

    # 5. Forbidden arcs
    model.addConstrs((x[i, i] == 0 for i in nodes), name="no_selfloop")
    model.addConstrs((x[fictitious_node, j] == 0 for j in nodes), name="no_from_fictitious")
    model.addConstrs((x[i, 0] == 0 for i in nodes if i != fictitious_node), name="no_to_depot0")

    # 6. Flow capacity
    model.addConstrs(
        (f[i, j] <= Q * x[i, j] for i in nodes for j in nodes if i != j),
        name="flow_capacity",
    )

    # 7. Flow conservation
    model.addConstrs(
        (
            gp.quicksum(f[i, j] for j in nodes if j != 0) - gp.quicksum(f[j, i] for j in nodes if j != fictitious_node)
            == Wit_before_dict[i] * g[i]
            for i in nodes_real
        ),
        name="flow_conservation",
    )

    # 8. MTZ subtour elimination
    u = model.addVars(nodes_real, lb=1, ub=n_bins, vtype=GRB.CONTINUOUS, name="u")
    model.addConstrs(
        (u[i] - u[j] + (n_bins + 1) * x[i, j] <= n_bins for i in nodes_real for j in nodes_real if i != j),
        name="MTZ",
    )

    # 9. Lazy subtour elimination callback
    def subtour_elimination_callback(model, where):
        """Dynamic subtour elimination during optimization"""
        if where == GRB.Callback.MIPSOL:
            x_val = model.cbGetSolution(model._x_vars)

            # Find active arcs among bins
            active_arcs = [(i, j) for (i, j), v in x_val.items() if v > 0.5 and i in nodes_real and j in nodes_real]

            # Find connected components using DFS
            visited = set()
            components = []

            for start_node in nodes_real:
                if start_node in visited:
                    continue

                component = set()
                stack = [start_node]

                while stack:
                    node = stack.pop()
                    if node in visited:
                        continue
                    visited.add(node)
                    component.add(node)
                    neighbors = [j for (i, j) in active_arcs if i == node and j not in visited]
                    stack.extend(neighbors)

                if len(component) > 0:
                    components.append(component)

            # Check each component for depot connection
            for comp in components:
                if len(comp) >= 2:
                    has_depot_entry = any(x_val.get((0, j), 0) > 0.5 for j in comp)
                    has_depot_exit = any(x_val.get((i, fictitious_node), 0) > 0.5 for i in comp)

                    if not (has_depot_entry or has_depot_exit):
                        # Subtour found! Add lazy cut
                        print(f"   🔍 Subtour detected: {len(comp)} nodes, adding cut...")
                        model.cbLazy(
                            gp.quicksum(
                                model._x_vars[(i, j)] for i in comp for j in comp if i != j and (i, j) in model._x_vars
                            )
                            <= len(comp) - 1
                        )

    model._x_vars = {(i, j): x[i, j] for i in nodes for j in nodes if i != j}

    # --------------------------
    # SOLVE
    print("\n" + "-" * 80)
    print("OPTIMIZING MODEL")
    print("-" * 80)

    start_time = time.time()
    model.optimize(subtour_elimination_callback)
    solve_time = time.time() - start_time

    print(f"✓ Optimization completed in {solve_time:.2f} seconds")
    print(f"  Status: {model.status}")
    if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
        print(f"  Objective: {model.objVal:.2f} €")
        print(f"  MIP Gap: {model.MIPGap * 100:.4f}%")

    if model.status == GRB.INFEASIBLE:
        print("❌ MODEL INFEASIBLE!")
        model.computeIIS()
        model.write(f"infeasible_day_{t}.ilp")
        continue

    # --------------------------
    # ROUTE RECONSTRUCTION
    print("\n" + "-" * 80)
    print("RECONSTRUCTING ROUTES")
    print("-" * 80)

    active_arcs = [(i, j) for i in nodes for j in nodes if i != j and hasattr(x[i, j], "X") and x[i, j].X > 0.5]
    bins_visited_g = [i for i in nodes_real if hasattr(g[i], "X") and g[i].X > 0.5]

    print(f"Active arcs: {len(active_arcs)}")
    print(f"Bins with g=1: {len(bins_visited_g)}")

    # Build adjacency
    adj_dict = {}
    for i, j in active_arcs:
        if i not in adj_dict:
            adj_dict[i] = []
        adj_dict[i].append(j)

    depot_starts = [j for (i, j) in active_arcs if i == 0 and j in nodes_real]
    print(f"Routes from depot: {len(depot_starts)}")

    vehicles_routes = []
    vehicles_metrics = []
    all_visited_in_routes = set()

    for veh_idx, start_bin in enumerate(depot_starts, 1):
        route = [0, start_bin]
        all_visited_in_routes.add(start_bin)
        current = start_bin
        visited_in_route = {0, start_bin}

        loop_guard = 0
        while current != fictitious_node and current != 0:
            loop_guard += 1
            if loop_guard > 500:
                print(f"  ⚠️ Loop detected at node {current}")
                break

            next_nodes = adj_dict.get(current, [])
            next_unvisited = [n for n in next_nodes if n not in visited_in_route or n == fictitious_node]

            if not next_unvisited:
                break

            next_node = next_unvisited[0]
            route.append(next_node)

            if next_node in nodes_real:
                all_visited_in_routes.add(next_node)
                visited_in_route.add(next_node)

            current = next_node

        vehicles_routes.append(route)

        # Calculate vehicle metrics
        bins_in_route = [n for n in route if n in nodes_real]
        dist_vehicle = sum(distances.get((route[i], route[i + 1]), 0.0) for i in range(len(route) - 1))
        kg_vehicle = sum(Wit_before_dict.get(b, 0.0) for b in bins_in_route)
        ratio_vehicle = (kg_vehicle / dist_vehicle) if dist_vehicle > 0 else 0

        # Load profile
        cumulative_load = 0.0
        max_load = 0.0
        for node in route:
            if node in nodes_real:
                cumulative_load += Wit_before_dict.get(node, 0.0)
                max_load = max(max_load, cumulative_load)
            elif node == 0 and route.index(node) > 0:  # Return to depot
                cumulative_load = 0.0

        capacity_util = (max_load / Q) * 100 if Q > 0 else 0

        vehicles_metrics.append(
            {
                "Day": t,
                "VehicleID": veh_idx,
                "Bins_visited": len(bins_in_route),
                "IDs_bins": bins_in_route,
                "Distance_km": dist_vehicle,
                "Kg_collected": kg_vehicle,
                "Ratio_kg_per_km": ratio_vehicle,
                "Max_load_kg": max_load,
                "Capacity_util_%": capacity_util,
                "Route": route,
            }
        )

        print(f"\n🚛 Vehicle {veh_idx}:")
        print(f"   Bins: {len(bins_in_route)}")
        print(f"   Distance: {dist_vehicle:.2f} km")
        print(f"   Kg: {kg_vehicle:.2f} kg")
        print(f"   Ratio: {ratio_vehicle:.2f} kg/km")
        print(f"   Max load: {max_load:.2f} kg ({capacity_util:.1f}%)")

    # --------------------------
    # VALIDATION
    missing_bins = [b for b in bins_visited_g if b not in all_visited_in_routes]

    if missing_bins:
        print(f"\n❌ SUBTOUR PROBLEM: {len(missing_bins)} bins missing")
        print(f"   Missing IDs: {missing_bins[:20]}...")
    else:
        print(f"\n✅ All {len(bins_visited_g)} bins in routes (no subtours)")

    # --------------------------
    # AGGREGATE METRICS

    num_vehicles = len(vehicles_metrics)
    total_dist = sum(vm["Distance_km"] for vm in vehicles_metrics)
    total_kg = sum(vm["Kg_collected"] for vm in vehicles_metrics)

    # Multiple ratio calculations
    ratio_global = (total_kg / total_dist) if total_dist > 0 else 0

    ratios = [vm["Ratio_kg_per_km"] for vm in vehicles_metrics if vm["Distance_km"] > 0]
    ratio_avg = (sum(ratios) / len(ratios)) if ratios else 0

    if total_dist > 0:
        ratio_weighted_dist = sum(
            vm["Ratio_kg_per_km"] * (vm["Distance_km"] / total_dist) for vm in vehicles_metrics if vm["Distance_km"] > 0
        )
    else:
        ratio_weighted_dist = 0

    if total_kg > 0:
        ratio_weighted_kg = sum(
            vm["Ratio_kg_per_km"] * (vm["Kg_collected"] / total_kg) for vm in vehicles_metrics if vm["Kg_collected"] > 0
        )
    else:
        ratio_weighted_kg = 0

    # Harmonic mean
    if len(ratios) > 0 and all(r > 0 for r in ratios):
        ratio_harmonic = len(ratios) / sum(1 / r for r in ratios)
    else:
        ratio_harmonic = 0

    # Dispersion
    if len(ratios) > 1:
        mean_r = ratio_avg
        variance = sum((r - mean_r) ** 2 for r in ratios) / len(ratios)
        std_dev = variance**0.5
        coef_var = (std_dev / mean_r * 100) if mean_r > 0 else 0
        range_r = max(ratios) - min(ratios)
    else:
        std_dev = 0
        coef_var = 0
        range_r = 0

    # Balance score
    if len(vehicles_metrics) > 1:
        kg_vals = [vm["Kg_collected"] for vm in vehicles_metrics]
        dist_vals = [vm["Distance_km"] for vm in vehicles_metrics]
        kg_imb = (max(kg_vals) - min(kg_vals)) / max(kg_vals) if max(kg_vals) > 0 else 0
        dist_imb = (max(dist_vals) - min(dist_vals)) / max(dist_vals) if max(dist_vals) > 0 else 0
        balance = (1 - (kg_imb + dist_imb) / 2) * 100
    else:
        kg_imb = 0
        dist_imb = 0
        balance = 100.0

    print("\n📊 EFFICIENCY METRICS:")
    print(f"   Global ratio: {ratio_global:.2f} kg/km")
    print(f"   Arithmetic mean: {ratio_avg:.2f} kg/km")
    print(f"   Coefficient of variation: {coef_var:.1f}%")
    print(f"   Balance score: {balance:.1f}/100")

    # --------------------------
    # UPDATE STATE
    for i in nodes_real:
        S_before = S_current[i]
        a_it = ait_kg_dict.get((i, t), 0.0)
        Wit_before = S_before + a_it
        collected_actual = 1 if g[i].X > 0.5 else 0
        S_after = 0.0 if collected_actual == 1 else Wit_before
        S_current[i] = S_after

        overflow_kg = max(0, Wit_before - E_dict[i])
        overflow_daily_rows.append({"Day": t, "BinID": i, "Overflow_kg": overflow_kg})

        profit_planned = R * Wit_before
        profit_actual = R * Wit_before if collected_actual == 1 else 0.0

        dynamic_rows.append(
            {
                "Day": t,
                "BinID": i,
                "S_before": S_before,
                "Daily_accumulation": a_it,
                "Waste_before": Wit_before,
                "Planned_collection": collected_planned_dict[i],
                "Actual_collection": collected_actual,
                "Waste_after": S_after,
                "Profit_planned": profit_planned,
                "Profit_actual": profit_actual,
                "InRoute": 1 if i in all_visited_in_routes else 0,
                "OverflowStatus": "AT_CAPACITY" if overflow_kg > 0 else "SAFE",
            }
        )

    # Export flows
    for i in nodes:
        for j in nodes:
            if i != j and hasattr(x[i, j], "X") and x[i, j].X > 0.5:
                flow_rows.append({"Day": t, "Var": "x", "From": i, "To": j, "Value": x[i, j].X})
    for i in nodes_real:
        if hasattr(g[i], "X") and g[i].X > 0.5:
            flow_rows.append({"Day": t, "Var": "g", "BinID": i, "Value": g[i].X})

    # Store vehicle details
    vehicle_details_rows.extend(vehicles_metrics)

    # KPI
    bins_not_visited = [i for i in nodes_real if i not in bins_visited_g]
    profit_total = R * total_kg
    vehicle_cost = Omega * len(vehicles_routes)
    FO = profit_total - C * total_dist - vehicle_cost
    overflow_bins = [i for i in nodes_real if Wit_before_dict[i] >= E_dict[i]]
    kg_overflow = sum(max(0, Wit_before_dict[i] - E_dict[i]) for i in overflow_bins)

    kpi_rows.append(
        {
            "Day": t,
            "CollectionNumber": ((t - FIRST_COLLECTION_DAY) // COLLECTION_FREQUENCY) + 1,
            "DaysSinceLastCollection": COLLECTION_FREQUENCY if t > FIRST_COLLECTION_DAY else 0,
            "Kg_collected": total_kg,
            "Total_distance_km": total_dist,
            "Revenue": profit_total,
            "Transport_cost": C * total_dist,
            "Vehicle_cost": vehicle_cost,
            "Objective": FO,
            "Num_vehicles": num_vehicles,
            "Bins_collected_g": len(bins_visited_g),
            "Bins_in_routes": len(all_visited_in_routes),
            "Bins_missing": len(missing_bins),
            "IDs_missing": missing_bins,
            "Bins_not_collected": len(bins_not_visited),
            "IDs_not_collected": bins_not_visited,
            # Efficiency ratios
            "Ratio_GLOBAL": ratio_global,
            "Ratio_ARITHMETIC": ratio_avg,
            "Ratio_WEIGHTED_DIST": ratio_weighted_dist,
            "Ratio_WEIGHTED_KG": ratio_weighted_kg,
            "Ratio_HARMONIC": ratio_harmonic,
            "Ratio_MIN": min(ratios) if ratios else 0,
            "Ratio_MAX": max(ratios) if ratios else 0,
            "Ratio_RANGE": range_r,
            "Ratio_STD_DEV": std_dev,
            "Ratio_COEF_VAR_%": coef_var,
            # Balance
            "Kg_imbalance_%": kg_imb * 100,
            "Dist_imbalance_%": dist_imb * 100,
            "Balance_score": balance,
            # Overflow
            "Overflow_kg": kg_overflow,
            "Num_bins_overflow": len(overflow_bins),
            "IDs_overflow": overflow_bins,
            # Solver
            "MIP_gap_%": model.MIPGap * 100,
            "Solve_time_s": solve_time,
            "Routes": vehicles_routes,
        }
    )

# --------------------------
# EXPORT
print("\n" + "=" * 80)
print(" " * 25 + "EXPORTING RESULTS")
print("=" * 80)

pd.DataFrame(dynamic_rows).to_excel(output_excel_dynamic, index=False)
pd.DataFrame(kpi_rows).to_excel(output_excel_kpi, index=False)
pd.DataFrame(flow_rows).to_excel(output_excel_flows, index=False)
pd.DataFrame(overflow_daily_rows).to_excel(output_excel_overflow, index=False)
pd.DataFrame(vehicle_details_rows).to_excel(output_excel_vehicles, index=False)

print(f"✅ Daily dynamics    → {output_excel_dynamic}")
print(f"✅ KPI summary       → {output_excel_kpi}")
print(f"✅ Flows             → {output_excel_flows}")
print(f"✅ Overflow tracking → {output_excel_overflow}")
print(f"✅ Vehicle details   → {output_excel_vehicles}")

# --------------------------
# FINAL SUMMARY
print("\n" + "=" * 80)
print(" " * 20 + "SIMULATION SUMMARY - 3-DAY POLICY")
print("=" * 80)

df_kpi = pd.DataFrame(kpi_rows)

if len(df_kpi) > 0:
    print("\n📅 COLLECTION SCHEDULE:")
    print(f"   Frequency: Every {COLLECTION_FREQUENCY} days")
    print(f"   First collection: Day {FIRST_COLLECTION_DAY}")
    print(f"   Collection days: {collection_days_schedule}")
    print(f"   Total collections in 30 days: {len(df_kpi)}")

    print("\n📊 AVERAGE PERFORMANCE PER COLLECTION DAY:")
    print(f"   Vehicles used: {df_kpi['Num_vehicles'].mean():.2f} ± {df_kpi['Num_vehicles'].std():.2f}")
    print(f"   Distance: {df_kpi['Total_distance_km'].mean():.2f} ± {df_kpi['Total_distance_km'].std():.2f} km")
    print(f"   Waste collected: {df_kpi['Kg_collected'].mean():.2f} ± {df_kpi['Kg_collected'].std():.2f} kg")
    print(f"   Profit: {df_kpi['Objective'].mean():.2f} ± {df_kpi['Objective'].std():.2f} €")

    print("\n📈 EFFICIENCY COMPARISON:")
    print(f"   Global ratio: {df_kpi['Ratio_GLOBAL'].mean():.2f} kg/km")
    print(f"   Arithmetic mean: {df_kpi['Ratio_ARITHMETIC'].mean():.2f} kg/km")
    print(f"   Harmonic mean: {df_kpi['Ratio_HARMONIC'].mean():.2f} kg/km")

    print("\n⚖️ BALANCE ANALYSIS:")
    print(f"   Average balance score: {df_kpi['Balance_score'].mean():.1f}/100")
    print(f"   Average CV: {df_kpi['Ratio_COEF_VAR_%'].mean():.1f}%")

    if df_kpi["Balance_score"].mean() > 80:
        print("   ✅ Well-balanced routes")
    else:
        print("   ⚠️ Imbalanced routes")

    total_missing = df_kpi["Bins_missing"].sum()
    if total_missing == 0:
        print("\n✅ NO SUBTOURS: All bins correctly routed")
    else:
        print(f"\n❌ SUBTOUR ISSUE: {total_missing} total missing bins")

print("=" * 80)

                         READING INPUT DATA
✓ Loaded 104 bins
✓ Loaded daily accumulation data
✓ Distance matrix loaded: 11236 arcs

                    COLLECTION SCHEDULE (3-DAY POLICY)

Collection will occur on 10 days:
Days: [1, 4, 7, 10, 13, 16, 19, 22, 25, 28]

Verification:
  Collection 1: Day 1 → Next: Day 4 (gap: 3 days)
  Collection 2: Day 4 → Next: Day 7 (gap: 3 days)
  Collection 3: Day 7 → Next: Day 10 (gap: 3 days)
  Collection 4: Day 10 → Next: Day 13 (gap: 3 days)
  Collection 5: Day 13 → Next: Day 16 (gap: 3 days)

                    STARTING 30-DAY SIMULATION

                                     DAY 1                                      
Waste status BEFORE collection:
  Total waste: 2508.00 kg
  Bins at/over capacity: 0/104

🗓️  COLLECTION DAY #1
   Days since last collection: 0
   Planned bins: 104 (ALL)

--------------------------------------------------------------------------------
BUILDING VRP MODEL
------------------------------------------------------------

In [2]:
# ============================================================
# DYNAMIC VRP — 30-DAY SIMULATION WITH 3-DAY COLLECTION POLICY
# CORRECTED VERSION: Collection every 3 days (1, 4, 7, ..., 28) + FINAL collection on Day 30
# Collection days: 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 30
# ============================================================

import time

import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# --------------------------
# FILE PATHS
# --------------------------
file_parameters = (
    r"C:\Users\hecto\OneDrive - Universidade de Lisboa\PhDtese\TestVRPP-Determinsit_104bins\parameters.xlsx"
)
file_distances = (
    r"C:\Users\hecto\OneDrive - Universidade de Lisboa\PhDtese\TestVRPP-Determinsit_104bins\distance_104bins.xlsx"
)

output_excel_kpi = (
    r"C:\Users\hecto\OneDrive - Universidade de Lisboa\PhDtese\TestVRPP-104bins_KPI_3DAY_POLICY_regular.xlsx"
)
output_excel_flows = output_excel_kpi.replace("_KPI_", "_Flows_")
output_excel_dynamic = output_excel_kpi.replace("_KPI_", "_DYNAMIC_")
output_excel_overflow = output_excel_kpi.replace("_KPI_", "_OVERFLOW_")
output_excel_vehicles = output_excel_kpi.replace("_KPI_", "_VEHICLE_DETAILS_")

# --------------------------
# MODEL PARAMETERS
# --------------------------
C = 1.0  # Transportation cost per km
R = 0.5837  # Revenue per kg of collected waste
Omega = 0.1  # Fixed cost per vehicle usage
Q = 3500.0  # Vehicle capacity (kg)
T_days = 30  # Simulation horizon (days)
T = list(range(1, T_days + 1))

# Waste container parameters
B = 19  # Container volume parameter
vol = 2.5  # Volume multiplier

# 🎯 COLLECTION POLICY PARAMETERS
COLLECTION_FREQUENCY = 3  # Base frequency: collect every 3 days
FIRST_COLLECTION_DAY = 1  # Start collecting on Day 1
LAST_DAY = T_days  # Ensure collection on last day

# Solver parameters
SOLVER_TIME_LIMIT = 28800  # 8 hours
SOLVER_MIP_GAP = 0.0001
SOLVER_THREADS = 4

# --------------------------
# READ INPUT DATA
# --------------------------
print("=" * 80)
print(" " * 25 + "READING INPUT DATA")
print("=" * 80)

df_params = pd.read_excel(file_parameters, sheet_name="Si_Ei", engine="openpyxl")
IDs_model = df_params["ID"].astype(int).tolist()
S_base = {int(r["ID"]): float(r["Si"]) for _, r in df_params.iterrows()}
E_dict = {int(r["ID"]): float(r["Ei"]) for _, r in df_params.iterrows()}
avg_ai = {int(r["ID"]): float(r["Mean"]) for _, r in df_params.iterrows()}

print(f"✓ Loaded {len(IDs_model)} bins")

df_ait = pd.read_excel(file_parameters, sheet_name="%ai_t", engine="openpyxl")
S_init = {}
ait_kg_dict = {}
for _, row in df_ait.iterrows():
    i = int(row["ID"])
    percent_S0 = float(row["Day 0"]) if "Day 0" in df_ait.columns and not pd.isna(row["Day 0"]) else 0.0
    S_init[i] = (percent_S0 / 100.0) * B * vol
    for t in T:
        col = f"Day {t}"
        ait_kg_dict[(i, t)] = (
            (float(row[col]) / 100.0 * B * vol) if col in df_ait.columns and not pd.isna(row[col]) else 0.0
        )

print("✓ Loaded daily accumulation data")

# Distance matrix
df_dist = pd.read_excel(file_distances, engine="openpyxl", index_col=0)
df_dist.index = df_dist.index.astype(int)
df_dist.columns = df_dist.columns.astype(int)

nodes_real = IDs_model
n_bins = len(nodes_real)
fictitious_node = n_bins + 1
nodes = [0] + nodes_real + [fictitious_node]

distances = {}
for i in nodes:
    for j in nodes:
        ii = 0 if i == fictitious_node else i
        jj = 0 if j == fictitious_node else j
        distances[(i, j)] = float(df_dist.loc[ii, jj])

print(f"✓ Distance matrix loaded: {len(distances)} arcs")

# --------------------------
# COMPUTE AND DISPLAY COLLECTION SCHEDULE
# --------------------------
print("\n" + "=" * 80)
print(" " * 15 + "COLLECTION SCHEDULE (3-DAY POLICY + FINAL DAY)")
print("=" * 80)

# Generate collection days: every 3 days starting from day 1
collection_days_schedule = [t for t in T if ((t - FIRST_COLLECTION_DAY) % COLLECTION_FREQUENCY == 0)]

# Add final day if not already included
if LAST_DAY not in collection_days_schedule:
    collection_days_schedule.append(LAST_DAY)
    collection_days_schedule.sort()

print(f"\n✅ Collection will occur on {len(collection_days_schedule)} days:")
print(f"   Days: {collection_days_schedule}")
print("\n📋 Schedule verification:")
for i in range(len(collection_days_schedule)):
    day = collection_days_schedule[i]
    if i > 0:
        gap = day - collection_days_schedule[i - 1]
        print(f"   Collection {i + 1}: Day {day:2d} (gap from previous: {gap} days)")
    else:
        print(f"   Collection {i + 1}: Day {day:2d} (FIRST)")

# Create a dictionary for quick lookup of days since last collection
days_since_last_collection = {}
for i, day in enumerate(collection_days_schedule):
    if i == 0:
        days_since_last_collection[day] = 0  # First collection
    else:
        days_since_last_collection[day] = day - collection_days_schedule[i - 1]

print("\n📊 Coverage verification:")
uncovered_days = [t for t in T if t not in collection_days_schedule]
max_gap = max(days_since_last_collection.values()) if days_since_last_collection else 0
print(f"   Days without collection: {len(uncovered_days)}/{len(T)}")
print(f"   Maximum gap between collections: {max_gap} days")
print(f"   Last collection day: {collection_days_schedule[-1]}")
if collection_days_schedule[-1] == T_days:
    print("   ✅ Simulation ends with collection (no residual waste)")
else:
    print(f"   ⚠️  Last {T_days - collection_days_schedule[-1]} days without collection")

# --------------------------
# INITIALIZE SIMULATION STATE
# --------------------------
S_current = {i: S_base[i] for i in nodes_real}

# Storage tables
dynamic_rows = []
kpi_rows = []
flow_rows = []
overflow_daily_rows = []
vehicle_details_rows = []

print("\n" + "=" * 80)
print(" " * 20 + "STARTING 30-DAY SIMULATION")
print("=" * 80)

# ==========================
# DAILY SIMULATION LOOP
# ==========================
for t in T:
    print(f"\n{'=' * 80}")
    print(f"{'DAY ' + str(t):^80}")
    print("=" * 80)

    # --------------------------
    # Compute waste before collection
    Wit_before_dict = {i: S_current[i] + ait_kg_dict.get((i, t), 0.0) for i in nodes_real}

    total_waste = sum(Wit_before_dict.values())
    bins_at_capacity = [i for i in nodes_real if Wit_before_dict[i] >= E_dict[i]]

    print("Waste status BEFORE collection:")
    print(f"  Total waste: {total_waste:.2f} kg")
    print(f"  Bins at/over capacity: {len(bins_at_capacity)}/{n_bins}")

    # --------------------------
    # 🎯 Determine if collection day using the schedule
    is_collection_day = t in collection_days_schedule

    collected_planned_dict = {i: 1 if is_collection_day else 0 for i in nodes_real}
    planned_bins = [i for i, v in collected_planned_dict.items() if v == 1]

    # Display schedule
    if is_collection_day:
        collection_number = collection_days_schedule.index(t) + 1
        days_since_last = days_since_last_collection[t]

        if t == LAST_DAY and t not in [d for d in collection_days_schedule[:-1]]:
            print(f"\n🗓️  FINAL COLLECTION (Day {t})")
            print(f"   Collection #{collection_number} of {len(collection_days_schedule)}")
        else:
            print(f"\n🗓️  COLLECTION DAY #{collection_number} of {len(collection_days_schedule)}")

        print(f"   Days since last collection: {days_since_last}")
        print(f"   Planned bins: {len(planned_bins)} (ALL)")
    else:
        # Find next collection day
        next_collections = [d for d in collection_days_schedule if d > t]
        days_until_next = next_collections[0] - t if next_collections else 0

        print("\n📅 Accumulation day (no collection)")
        print(f"   Days until next collection: {days_until_next}")

    # --------------------------
    # NON-COLLECTION DAYS: Accumulate only
    if not is_collection_day:
        for i in nodes_real:
            S_before = S_current[i]
            a_it = ait_kg_dict.get((i, t), 0.0)
            Wit_before = S_before + a_it
            S_after = Wit_before
            S_current[i] = S_after

            overflow_kg = max(0, Wit_before - E_dict[i])
            overflow_daily_rows.append({"Day": t, "BinID": i, "Overflow_kg": overflow_kg})

            dynamic_rows.append(
                {
                    "Day": t,
                    "BinID": i,
                    "S_before": S_before,
                    "Daily_accumulation": a_it,
                    "Waste_before": Wit_before,
                    "Planned_collection": 0,
                    "Actual_collection": 0,
                    "Waste_after": S_after,
                    "Profit_planned": 0.0,
                    "Profit_actual": 0.0,
                    "InRoute": 0,
                    "OverflowStatus": "AT_CAPACITY" if overflow_kg > 0 else "SAFE",
                }
            )
        continue

    # --------------------------
    # COLLECTION DAY: Build VRP model
    print("\n" + "-" * 80)
    print("BUILDING VRP MODEL")
    print("-" * 80)

    model = gp.Model(f"VRP_Day_{t}")
    model.setParam("OutputFlag", 0)
    model.setParam("TimeLimit", SOLVER_TIME_LIMIT)
    model.setParam("MIPGap", SOLVER_MIP_GAP)
    model.setParam("Threads", SOLVER_THREADS)
    model.setParam("LazyConstraints", 1)

    # Decision variables
    x = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
    g = model.addVars(nodes_real, vtype=GRB.BINARY, name="g")
    f = model.addVars(nodes, nodes, vtype=GRB.CONTINUOUS, lb=0, name="f")

    # Objective
    model.setObjective(
        R * gp.quicksum(Wit_before_dict[i] * g[i] for i in nodes_real)
        - C * gp.quicksum(x[i, j] * distances[(i, j)] for i in nodes for j in nodes if i != j)
        - Omega * gp.quicksum(x[0, j] for j in nodes_real),
        GRB.MAXIMIZE,
    )

    # --------------------------
    # CONSTRAINTS

    # 1. Collect ALL bins on collection days
    model.addConstrs((g[i] == 1 for i in nodes_real), name="collect_all")

    # 2. Inflow: exactly ONE arc entering each visited bin (not from fictitious_node)
    model.addConstrs(
        (gp.quicksum(x[j, i] for j in nodes if j != i and j != fictitious_node) == g[i] for i in nodes_real),
        name="inflow",
    )

    # 3. Outflow: exactly ONE arc leaving each visited bin (not to depot 0)
    model.addConstrs(
        (gp.quicksum(x[i, j] for j in nodes if j != i and j != 0) == g[i] for i in nodes_real),
        name="outflow",
    )

    # 4. Depot balance
    model.addConstr(
        gp.quicksum(x[0, j] for j in nodes_real) == gp.quicksum(x[i, fictitious_node] for i in nodes_real),
        name="depot_balance",
    )

    # 5. Forbidden arcs
    model.addConstrs((x[i, i] == 0 for i in nodes), name="no_selfloop")
    model.addConstrs((x[fictitious_node, j] == 0 for j in nodes), name="no_from_fictitious")
    model.addConstrs((x[i, 0] == 0 for i in nodes if i != fictitious_node), name="no_to_depot0")

    # 6. Flow capacity
    model.addConstrs(
        (f[i, j] <= Q * x[i, j] for i in nodes for j in nodes if i != j),
        name="flow_capacity",
    )

    # 7. Flow conservation
    model.addConstrs(
        (
            gp.quicksum(f[i, j] for j in nodes if j != 0) - gp.quicksum(f[j, i] for j in nodes if j != fictitious_node)
            == Wit_before_dict[i] * g[i]
            for i in nodes_real
        ),
        name="flow_conservation",
    )

    # 8. MTZ subtour elimination
    u = model.addVars(nodes_real, lb=1, ub=n_bins, vtype=GRB.CONTINUOUS, name="u")
    model.addConstrs(
        (u[i] - u[j] + (n_bins + 1) * x[i, j] <= n_bins for i in nodes_real for j in nodes_real if i != j),
        name="MTZ",
    )

    # 9. Lazy subtour elimination callback
    def subtour_elimination_callback(model, where):
        """Dynamic subtour elimination during optimization"""
        if where == GRB.Callback.MIPSOL:
            x_val = model.cbGetSolution(model._x_vars)

            # Find active arcs among bins
            active_arcs = [(i, j) for (i, j), v in x_val.items() if v > 0.5 and i in nodes_real and j in nodes_real]

            # Find connected components using DFS
            visited = set()
            components = []

            for start_node in nodes_real:
                if start_node in visited:
                    continue

                component = set()
                stack = [start_node]

                while stack:
                    node = stack.pop()
                    if node in visited:
                        continue
                    visited.add(node)
                    component.add(node)
                    neighbors = [j for (i, j) in active_arcs if i == node and j not in visited]
                    stack.extend(neighbors)

                if len(component) > 0:
                    components.append(component)

            # Check each component for depot connection
            for comp in components:
                if len(comp) >= 2:
                    has_depot_entry = any(x_val.get((0, j), 0) > 0.5 for j in comp)
                    has_depot_exit = any(x_val.get((i, fictitious_node), 0) > 0.5 for i in comp)

                    if not (has_depot_entry or has_depot_exit):
                        # Subtour found! Add lazy cut
                        print(f"   🔍 Subtour detected: {len(comp)} nodes, adding cut...")
                        model.cbLazy(
                            gp.quicksum(
                                model._x_vars[(i, j)] for i in comp for j in comp if i != j and (i, j) in model._x_vars
                            )
                            <= len(comp) - 1
                        )

    model._x_vars = {(i, j): x[i, j] for i in nodes for j in nodes if i != j}

    # --------------------------
    # SOLVE
    print("\n" + "-" * 80)
    print("OPTIMIZING MODEL")
    print("-" * 80)

    start_time = time.time()
    model.optimize(subtour_elimination_callback)
    solve_time = time.time() - start_time

    print(f"✓ Optimization completed in {solve_time:.2f} seconds")
    print(f"  Status: {model.status}")
    if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
        print(f"  Objective: {model.objVal:.2f} €")
        print(f"  MIP Gap: {model.MIPGap * 100:.4f}%")

    if model.status == GRB.INFEASIBLE:
        print("❌ MODEL INFEASIBLE!")
        model.computeIIS()
        model.write(f"infeasible_day_{t}.ilp")
        continue

    # --------------------------
    # ROUTE RECONSTRUCTION
    print("\n" + "-" * 80)
    print("RECONSTRUCTING ROUTES")
    print("-" * 80)

    active_arcs = [(i, j) for i in nodes for j in nodes if i != j and hasattr(x[i, j], "X") and x[i, j].X > 0.5]
    bins_visited_g = [i for i in nodes_real if hasattr(g[i], "X") and g[i].X > 0.5]

    print(f"Active arcs: {len(active_arcs)}")
    print(f"Bins with g=1: {len(bins_visited_g)}")

    # Build adjacency
    adj_dict = {}
    for i, j in active_arcs:
        if i not in adj_dict:
            adj_dict[i] = []
        adj_dict[i].append(j)

    depot_starts = [j for (i, j) in active_arcs if i == 0 and j in nodes_real]
    print(f"Routes from depot: {len(depot_starts)}")

    vehicles_routes = []
    vehicles_metrics = []
    all_visited_in_routes = set()

    for veh_idx, start_bin in enumerate(depot_starts, 1):
        route = [0, start_bin]
        all_visited_in_routes.add(start_bin)
        current = start_bin
        visited_in_route = {0, start_bin}

        loop_guard = 0
        while current != fictitious_node and current != 0:
            loop_guard += 1
            if loop_guard > 500:
                print(f"  ⚠️ Loop detected at node {current}")
                break

            next_nodes = adj_dict.get(current, [])
            next_unvisited = [n for n in next_nodes if n not in visited_in_route or n == fictitious_node]

            if not next_unvisited:
                break

            next_node = next_unvisited[0]
            route.append(next_node)

            if next_node in nodes_real:
                all_visited_in_routes.add(next_node)
                visited_in_route.add(next_node)

            current = next_node

        vehicles_routes.append(route)

        # Calculate vehicle metrics
        bins_in_route = [n for n in route if n in nodes_real]
        dist_vehicle = sum(distances.get((route[i], route[i + 1]), 0.0) for i in range(len(route) - 1))
        kg_vehicle = sum(Wit_before_dict.get(b, 0.0) for b in bins_in_route)
        ratio_vehicle = (kg_vehicle / dist_vehicle) if dist_vehicle > 0 else 0

        # Load profile
        cumulative_load = 0.0
        max_load = 0.0
        for node in route:
            if node in nodes_real:
                cumulative_load += Wit_before_dict.get(node, 0.0)
                max_load = max(max_load, cumulative_load)
            elif node == 0 and route.index(node) > 0:  # Return to depot
                cumulative_load = 0.0

        capacity_util = (max_load / Q) * 100 if Q > 0 else 0

        vehicles_metrics.append(
            {
                "Day": t,
                "VehicleID": veh_idx,
                "Bins_visited": len(bins_in_route),
                "IDs_bins": bins_in_route,
                "Distance_km": dist_vehicle,
                "Kg_collected": kg_vehicle,
                "Ratio_kg_per_km": ratio_vehicle,
                "Max_load_kg": max_load,
                "Capacity_util_%": capacity_util,
                "Route": route,
            }
        )

        print(f"\n🚛 Vehicle {veh_idx}:")
        print(f"   Bins: {len(bins_in_route)}")
        print(f"   Distance: {dist_vehicle:.2f} km")
        print(f"   Kg: {kg_vehicle:.2f} kg")
        print(f"   Ratio: {ratio_vehicle:.2f} kg/km")
        print(f"   Max load: {max_load:.2f} kg ({capacity_util:.1f}%)")

    # --------------------------
    # VALIDATION
    missing_bins = [b for b in bins_visited_g if b not in all_visited_in_routes]

    if missing_bins:
        print(f"\n❌ SUBTOUR PROBLEM: {len(missing_bins)} bins missing")
        print(f"   Missing IDs: {missing_bins[:20]}...")
    else:
        print(f"\n✅ All {len(bins_visited_g)} bins in routes (no subtours)")

    # --------------------------
    # AGGREGATE METRICS

    num_vehicles = len(vehicles_metrics)
    total_dist = sum(vm["Distance_km"] for vm in vehicles_metrics)
    total_kg = sum(vm["Kg_collected"] for vm in vehicles_metrics)

    # Multiple ratio calculations
    ratio_global = (total_kg / total_dist) if total_dist > 0 else 0

    ratios = [vm["Ratio_kg_per_km"] for vm in vehicles_metrics if vm["Distance_km"] > 0]
    ratio_avg = (sum(ratios) / len(ratios)) if ratios else 0

    if total_dist > 0:
        ratio_weighted_dist = sum(
            vm["Ratio_kg_per_km"] * (vm["Distance_km"] / total_dist) for vm in vehicles_metrics if vm["Distance_km"] > 0
        )
    else:
        ratio_weighted_dist = 0

    if total_kg > 0:
        ratio_weighted_kg = sum(
            vm["Ratio_kg_per_km"] * (vm["Kg_collected"] / total_kg) for vm in vehicles_metrics if vm["Kg_collected"] > 0
        )
    else:
        ratio_weighted_kg = 0

    # Harmonic mean
    if len(ratios) > 0 and all(r > 0 for r in ratios):
        ratio_harmonic = len(ratios) / sum(1 / r for r in ratios)
    else:
        ratio_harmonic = 0

    # Dispersion
    if len(ratios) > 1:
        mean_r = ratio_avg
        variance = sum((r - mean_r) ** 2 for r in ratios) / len(ratios)
        std_dev = variance**0.5
        coef_var = (std_dev / mean_r * 100) if mean_r > 0 else 0
        range_r = max(ratios) - min(ratios)
    else:
        std_dev = 0
        coef_var = 0
        range_r = 0

    # Balance score
    if len(vehicles_metrics) > 1:
        kg_vals = [vm["Kg_collected"] for vm in vehicles_metrics]
        dist_vals = [vm["Distance_km"] for vm in vehicles_metrics]
        kg_imb = (max(kg_vals) - min(kg_vals)) / max(kg_vals) if max(kg_vals) > 0 else 0
        dist_imb = (max(dist_vals) - min(dist_vals)) / max(dist_vals) if max(dist_vals) > 0 else 0
        balance = (1 - (kg_imb + dist_imb) / 2) * 100
    else:
        kg_imb = 0
        dist_imb = 0
        balance = 100.0

    print("\n📊 EFFICIENCY METRICS:")
    print(f"   Global ratio: {ratio_global:.2f} kg/km")
    print(f"   Arithmetic mean: {ratio_avg:.2f} kg/km")
    print(f"   Coefficient of variation: {coef_var:.1f}%")
    print(f"   Balance score: {balance:.1f}/100")

    # --------------------------
    # UPDATE STATE
    for i in nodes_real:
        S_before = S_current[i]
        a_it = ait_kg_dict.get((i, t), 0.0)
        Wit_before = S_before + a_it
        collected_actual = 1 if g[i].X > 0.5 else 0
        S_after = 0.0 if collected_actual == 1 else Wit_before
        S_current[i] = S_after

        overflow_kg = max(0, Wit_before - E_dict[i])
        overflow_daily_rows.append({"Day": t, "BinID": i, "Overflow_kg": overflow_kg})

        profit_planned = R * Wit_before
        profit_actual = R * Wit_before if collected_actual == 1 else 0.0

        dynamic_rows.append(
            {
                "Day": t,
                "BinID": i,
                "S_before": S_before,
                "Daily_accumulation": a_it,
                "Waste_before": Wit_before,
                "Planned_collection": collected_planned_dict[i],
                "Actual_collection": collected_actual,
                "Waste_after": S_after,
                "Profit_planned": profit_planned,
                "Profit_actual": profit_actual,
                "InRoute": 1 if i in all_visited_in_routes else 0,
                "OverflowStatus": "AT_CAPACITY" if overflow_kg > 0 else "SAFE",
            }
        )

    # Export flows
    for i in nodes:
        for j in nodes:
            if i != j and hasattr(x[i, j], "X") and x[i, j].X > 0.5:
                flow_rows.append({"Day": t, "Var": "x", "From": i, "To": j, "Value": x[i, j].X})
    for i in nodes_real:
        if hasattr(g[i], "X") and g[i].X > 0.5:
            flow_rows.append({"Day": t, "Var": "g", "BinID": i, "Value": g[i].X})

    # Store vehicle details
    vehicle_details_rows.extend(vehicles_metrics)

    # KPI
    bins_not_visited = [i for i in nodes_real if i not in bins_visited_g]
    profit_total = R * total_kg
    vehicle_cost = Omega * len(vehicles_routes)
    FO = profit_total - C * total_dist - vehicle_cost
    overflow_bins = [i for i in nodes_real if Wit_before_dict[i] >= E_dict[i]]
    kg_overflow = sum(max(0, Wit_before_dict[i] - E_dict[i]) for i in overflow_bins)

    kpi_rows.append(
        {
            "Day": t,
            "CollectionNumber": collection_days_schedule.index(t) + 1,
            "DaysSinceLastCollection": days_since_last_collection[t],
            "Kg_collected": total_kg,
            "Total_distance_km": total_dist,
            "Revenue": profit_total,
            "Transport_cost": C * total_dist,
            "Vehicle_cost": vehicle_cost,
            "Objective": FO,
            "Num_vehicles": num_vehicles,
            "Bins_collected_g": len(bins_visited_g),
            "Bins_in_routes": len(all_visited_in_routes),
            "Bins_missing": len(missing_bins),
            "IDs_missing": missing_bins,
            "Bins_not_collected": len(bins_not_visited),
            "IDs_not_collected": bins_not_visited,
            # Efficiency ratios
            "Ratio_GLOBAL": ratio_global,
            "Ratio_ARITHMETIC": ratio_avg,
            "Ratio_WEIGHTED_DIST": ratio_weighted_dist,
            "Ratio_WEIGHTED_KG": ratio_weighted_kg,
            "Ratio_HARMONIC": ratio_harmonic,
            "Ratio_MIN": min(ratios) if ratios else 0,
            "Ratio_MAX": max(ratios) if ratios else 0,
            "Ratio_RANGE": range_r,
            "Ratio_STD_DEV": std_dev,
            "Ratio_COEF_VAR_%": coef_var,
            # Balance
            "Kg_imbalance_%": kg_imb * 100,
            "Dist_imbalance_%": dist_imb * 100,
            "Balance_score": balance,
            # Overflow
            "Overflow_kg": kg_overflow,
            "Num_bins_overflow": len(overflow_bins),
            "IDs_overflow": overflow_bins,
            # Solver
            "MIP_gap_%": model.MIPGap * 100,
            "Solve_time_s": solve_time,
            "Routes": vehicles_routes,
        }
    )

# --------------------------
# EXPORT
print("\n" + "=" * 80)
print(" " * 25 + "EXPORTING RESULTS")
print("=" * 80)

pd.DataFrame(dynamic_rows).to_excel(output_excel_dynamic, index=False)
pd.DataFrame(kpi_rows).to_excel(output_excel_kpi, index=False)
pd.DataFrame(flow_rows).to_excel(output_excel_flows, index=False)
pd.DataFrame(overflow_daily_rows).to_excel(output_excel_overflow, index=False)
pd.DataFrame(vehicle_details_rows).to_excel(output_excel_vehicles, index=False)

print(f"✅ Daily dynamics    → {output_excel_dynamic}")
print(f"✅ KPI summary       → {output_excel_kpi}")
print(f"✅ Flows             → {output_excel_flows}")
print(f"✅ Overflow tracking → {output_excel_overflow}")
print(f"✅ Vehicle details   → {output_excel_vehicles}")

# --------------------------
# FINAL SUMMARY
print("\n" + "=" * 80)
print(" " * 15 + "SIMULATION SUMMARY - 3-DAY POLICY + FINAL DAY")
print("=" * 80)

df_kpi = pd.DataFrame(kpi_rows)

if len(df_kpi) > 0:
    print("\n📅 COLLECTION SCHEDULE:")
    print(f"   Base frequency: Every {COLLECTION_FREQUENCY} days")
    print(f"   First collection: Day {FIRST_COLLECTION_DAY}")
    print(f"   Final collection: Day {LAST_DAY}")
    print(f"   Collection days: {collection_days_schedule}")
    print(f"   Total collections in 30 days: {len(df_kpi)}")

    print("\n📊 AVERAGE PERFORMANCE PER COLLECTION DAY:")
    print(f"   Vehicles used: {df_kpi['Num_vehicles'].mean():.2f} ± {df_kpi['Num_vehicles'].std():.2f}")
    print(f"   Distance: {df_kpi['Total_distance_km'].mean():.2f} ± {df_kpi['Total_distance_km'].std():.2f} km")
    print(f"   Waste collected: {df_kpi['Kg_collected'].mean():.2f} ± {df_kpi['Kg_collected'].std():.2f} kg")
    print(f"   Profit: {df_kpi['Objective'].mean():.2f} ± {df_kpi['Objective'].std():.2f} €")

    print("\n📈 EFFICIENCY COMPARISON:")
    print(f"   Global ratio: {df_kpi['Ratio_GLOBAL'].mean():.2f} kg/km")
    print(f"   Arithmetic mean: {df_kpi['Ratio_ARITHMETIC'].mean():.2f} kg/km")
    print(f"   Harmonic mean: {df_kpi['Ratio_HARMONIC'].mean():.2f} kg/km")

    print("\n⚖️ BALANCE ANALYSIS:")
    print(f"   Average balance score: {df_kpi['Balance_score'].mean():.1f}/100")
    print(f"   Average CV: {df_kpi['Ratio_COEF_VAR_%'].mean():.1f}%")

    if df_kpi["Balance_score"].mean() > 80:
        print("   ✅ Well-balanced routes")
    else:
        print("   ⚠️ Imbalanced routes")

    # Analyze gap between collections
    print("\n📏 COLLECTION INTERVAL ANALYSIS:")
    gaps = df_kpi["DaysSinceLastCollection"].values
    print(f"   Minimum gap: {gaps.min()} days")
    print(f"   Maximum gap: {gaps.max()} days")
    print(f"   Average gap: {gaps[1:].mean():.2f} days")  # Skip first (gap=0)

    # Check final state
    df_dynamic = pd.DataFrame(dynamic_rows)
    final_day_data = df_dynamic[df_dynamic["Day"] == T_days]
    total_final_waste = final_day_data["Waste_after"].sum()

    print(f"\n🎯 FINAL STATE (End of Day {T_days}):")
    print(f"   Total residual waste: {total_final_waste:.2f} kg")
    if total_final_waste < 1.0:
        print("   ✅ COMPLETE COVERAGE: No residual waste at end of simulation")
    else:
        print("   ⚠️  Residual waste remaining")

    total_missing = df_kpi["Bins_missing"].sum()
    if total_missing == 0:
        print("\n✅ NO SUBTOURS: All bins correctly routed in all collections")
    else:
        print(f"\n❌ SUBTOUR ISSUE: {total_missing} total missing bins across all days")

print("=" * 80)
print(" " * 20 + "SIMULATION COMPLETED SUCCESSFULLY")
print("=" * 80)

                         READING INPUT DATA
✓ Loaded 104 bins
✓ Loaded daily accumulation data
✓ Distance matrix loaded: 11236 arcs

               COLLECTION SCHEDULE (3-DAY POLICY + FINAL DAY)

✅ Collection will occur on 11 days:
   Days: [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 30]

📋 Schedule verification:
   Collection 1: Day  1 (FIRST)
   Collection 2: Day  4 (gap from previous: 3 days)
   Collection 3: Day  7 (gap from previous: 3 days)
   Collection 4: Day 10 (gap from previous: 3 days)
   Collection 5: Day 13 (gap from previous: 3 days)
   Collection 6: Day 16 (gap from previous: 3 days)
   Collection 7: Day 19 (gap from previous: 3 days)
   Collection 8: Day 22 (gap from previous: 3 days)
   Collection 9: Day 25 (gap from previous: 3 days)
   Collection 10: Day 28 (gap from previous: 3 days)
   Collection 11: Day 30 (gap from previous: 2 days)

📊 Coverage verification:
   Days without collection: 19/30
   Maximum gap between collections: 3 days
   Last collection day: 30
   ✅ Si